原生形式lightgbm(import lightgbm as lgb)

In [1]:
# 从sklearn 调入所需要的包
from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
#导入鸢尾花数据
iris = datasets.load_iris()
data = iris.data
label = iris.target


In [2]:
print(data)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [3]:
print(pd.DataFrame(data).head())
print(pd.DataFrame(label).head())
data1 = pd.DataFrame(data)
## 花萼长宽花瓣长宽
data1.columns = ['sepal_l','sepal_w','petal_l','petal_w']
print(data1.head())
label1 =pd.DataFrame(label)
label1.columns=['label']
print(label1.head())
#注意这里data label顺序是一致的，千万别打乱

     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0
   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2
   label
0      0
1      0
2      0
3      0
4      0


In [4]:
# 划分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
print("训练集长度:", len(train_x))
print("测试集长度:", len(test_x))

训练集长度: 105
测试集长度: 45


lightgbm原生态接口

In [5]:
# 转换为DMatrix数据格式
import lightgbm as lgb
train_data = lgb.Dataset(train_x,train_y)
test_data = lgb.Dataset(test_x,test_y)
# 设置参数
lgb_params = {
   'boosting_type': 'gbdt',  
    'objective': 'multiclass',
    'metric': 'multi_error', 
    'verbose': 1 , # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
     'num_class':3 #lightgbm.basic.LightGBMError: b'Number of classes should be specified and greater than 1 for multiclass training'
    }

# 模型训练
clf = lgb.train(lgb_params,train_data,num_boost_round =10,
                valid_sets = [train_data,test_data], 
                verbose_eval = 1)
# 模型预测
test_pre = clf.predict(test_x, num_iteration=clf.best_iteration)
# print(test_pre)
print(test_pre[:5])

# 选择表示最高概率的列
test_pre_1 = np.asarray([np.argmax(row) for row in test_pre])
print("test的预测结果:",test_pre_1)

# 模型评估
print('验证集精准率：',precision_score(test_y, test_pre_1, average='macro')) 
print('验证集召回率：',recall_score(test_y, test_pre_1, average='macro'))  

[1]	training's multi_error: 0.133333	valid_1's multi_error: 0.155556
[2]	training's multi_error: 0.0571429	valid_1's multi_error: 0.0444444
[3]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[4]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[5]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[6]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[7]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[8]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[9]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[10]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[[0.13683286 0.63500393 0.22816321]
 [0.69436834 0.15467706 0.15095461]
 [0.12934308 0.16125127 0.70940565]
 [0.14172417 0.62195656 0.23631927]
 [0.13683286 0.63500393 0.22816321]]
test的预测结果: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


Sklearn接口形式使用lightgbm

In [6]:
import lightgbm as lgb
lgb_params = {
    'learning_rate':0.1,
#     'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,  
    'objective':'multiclass',
    'n_estimators':300
}
model=lgb.LGBMClassifier(**lgb_params)

model.fit(train_x,train_y)
# 预测
#输出预测结果
test_pre2 = model.predict(test_x)
print(test_pre2)
# 模型评估
print('验证集精准率：',precision_score(test_y, test_pre2, average='macro')) 
print('验证集召回率：',recall_score(test_y, test_pre2, average='macro'))  

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


总结：
1. lgb.train中正则化参数为"lambda_l1", "lambda_l1"，sklearn中则为'reg_alpha', 'reg_lambda'。
2. 多分类时lgb.train除了'objective':'multiclass',还要指定"num_class":5，而sklearn接口只需要指定'objective':'multiclass'。
3. 迭代次数在sklearn中是'n_estimators':20，在初始化模型时指定
